In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-04-18 07:26:38--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.26
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.26|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-04-18 07:26:38--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1594 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.2 and Spark NLP 3.0.1
-                   100%[====

In [ ]:
# import nlu

# pipe = nlu.load('spell')

spellcheck_dl download started this may take some time.
Approximate size to download 112.2 MB
[OK!]


In [ ]:
# import pandas as pd

# df = pd.read_csv('/content/drive/MyDrive/sentiment data/train_sent_final_with_folds.csv')

In [ ]:
def spell(x):
  return ' '.join(pipe.predict(x,output_level='sentence')['spell'][0])

In [ ]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.types import StringType,StructType,StructField,IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import json
from sklearn import model_selection
import time
class BuildModel:
  def __init__(self,path):
    self.spark = sparknlp.start()
    self.init_prep(path)
    # self.prepare_data(path)
    self.load_pipe_components()
    self.create_fin_pipeline()
    self.evaluation()
    
  def process(self):
    l = []
    for i in range(len(self.fin_tfolds)):
      start = time.time()
      print('ROUND '+str(i)+' started')
      l.append(self.run_training(self.fin_tfolds[i],self.fin_vfolds[i],i))
      end = time.time()-start
      print('ROUND '+str(i)+' completed, Time taken: '+str(end)+' seconds')
      print('\n')
    return l  

  def init_prep(self,path):
    df = pd.read_csv(path)
    df_vp = df[df['final_sent_class'] == 'VPos'].sample(n=1000)
    df_p = df[df['final_sent_class'] == 'Pos'].sample(n=1000)
    df_neu = df[df['final_sent_class'] == 'Neu'].sample(n=1000)
    df_n = df[df['final_sent_class'] == 'Neg'].sample(n=1000)
    df_vn = df[df['final_sent_class'] == 'VNeg'].sample(n=1000)

    df = pd.concat([df_vp,df_p,df_neu,df_n,df_vn],ignore_index=True).sample(frac=1).reset_index(drop=True)
    df = df.drop(['kfold','final_sent_class'],axis=1)
    skf = model_selection.StratifiedKFold(n_splits = 5)

    for f, (t_,v_) in enumerate(skf.split(X=df,y=df.sentiment.values)):
        df.loc[v_,"kfold"] = f


    tfolds = [df[df.kfold != fold].drop('kfold',axis=1).reset_index(drop=True) for fold in range(5)]
    vfolds = [df[df.kfold == fold].drop('kfold',axis=1).reset_index(drop=True) for fold in range(5)]
    schema = StructType([StructField('id', StringType(), True),                     
                      StructField('text', StringType(), True),
                      StructField('label', IntegerType(), True)])
                      
    self.fin_tfolds = [self.spark.createDataFrame(data,schema) for data in tfolds]
    self.fin_vfolds = [self.spark.createDataFrame(data,schema) for data in vfolds]



  # def prepare_data(self,path):
  #   df = pd.read_csv(path)
  #   if 'sarcasm' in list(df.columns):
  #     df = df.drop('sarcasm',axis=1)
  #     df = df.rename({'sar_num':'target'})
  #   elif 'final_sent_class' in list(df.columns):
  #     df = df.drop('final_sent_class',axis=1)
  #     df = df.rename({'sentiment':'target'})
          
  #   tfolds = [df[df.kfold != fold].drop('kfold',axis=1).reset_index(drop=True) for fold in range(5)]
  #   vfolds = [df[df.kfold == fold].drop('kfold',axis=1).reset_index(drop=True) for fold in range(5)]
  #   schema = StructType([StructField('id', StringType(), True),                     
  #                     StructField('text', StringType(), True),
  #                     StructField('label', IntegerType(), True)])
                      
  #   self.fin_tfolds = [self.spark.createDataFrame(data,schema) for data in tfolds]
  #   self.fin_vfolds = [self.spark.createDataFrame(data,schema) for data in vfolds]
    
  def load_pipe_components(self):
    self.document = DocumentAssembler().setInputCol("text").setOutputCol("document")
    # self.tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")
    # self.spellModel = ContextSpellCheckerModel.pretrained().setInputCols("token").setOutputCol("spell")
    # self.lemmatizer = LemmatizerModel.pretrained(name="lemma_antbnc").setInputCols(["spell"]).setOutputCol("lemma").setLazyAnnotator(False)
    self.sentenceDetector = SentenceDetector().setInputCols(["document"]).setOutputCol("sentence")
    # self.use = UniversalSentenceEncoder.pretrained().setInputCols(["sentence"]).setOutputCol("embeddings")
    self.embeddings = BertSentenceEmbeddings.pretrained("sent_small_bert_L6_768", "en").setInputCols("sentence").setOutputCol("embeddings")
    # self.embeddings = BertEmbeddings.pretrained("small_bert_L12_128", "en").setInputCols("sentence", "lemma").setOutputCol("embeddings")
    # self.sen_emb = SentenceEmbeddings().setInputCols(["document","embeddings"]).setOutputCol("sen_emb").setPoolingStrategy("AVERAGE")
    self.embeddings_finisher = EmbeddingsFinisher().setInputCols("embeddings").setOutputCols("embeddings_vectors").setCleanAnnotations(True).setOutputAsVector(True)
    # self.assembler = VectorAssembler(inputCols=['embeddings_vectors'],outputCol='features')
    self.ml = LogisticRegression(featuresCol='input',labelCol='label')
    
    
    #vec_assembler
    #ml model

  def create_fin_pipeline(self):
    
    self.nlp_pipeline = Pipeline(stages = [
          self.document,
          # self.tokenizer,
          # self.spellModel,
          # self.lemmatizer,
          self.sentenceDetector,
          # self.use,
          self.embeddings,
          # self.sen_emb,
          self.embeddings_finisher
          # self.assembler,
          ])
    self.ml_pipeline = Pipeline(stages = [self.ml])
          
  def run_training(self,df_tr,df_val,i):
    df_tr_emb = self.nlp_pipeline.fit(df_tr).transform(df_tr)
    df_tr_emb = df_tr_emb.select('id',F.explode('embeddings_vectors').alias('input'),'label')
    self.model = self.ml_pipeline.fit(df_tr_emb)
    df_val_emb = self.nlp_pipeline.fit(df_val).transform(df_val)
    df_val_emb = df_val_emb.select('id',F.explode('embeddings_vectors').alias('input'))
    df_res = self.model.transform(df_val_emb)
    df_eval = df_res.join(df_val,on='id',how='inner').select('label','prediction')
    acc = self.evaluator.evaluate(df_eval, {self.evaluator.metricName: "accuracy"})
    f1 = self.evaluator.evaluate(df_eval, {self.evaluator.metricName: "f1"})
    weightedPrecision = self.evaluator.evaluate(df_eval, {self.evaluator.metricName: "weightedPrecision"})
    weightedRecall = self.evaluator.evaluate(df_eval, {self.evaluator.metricName: "weightedRecall"})

    return {'accuracy': acc,'f1':f1,'weighted precision': weightedPrecision,'weighted recall':weightedRecall}
    
  def evaluation(self):
    self.evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
    
  def save(self):
    return self.model

In [ ]:
bm = BuildModel('/content/drive/MyDrive/sentiment data/train_sent_final_with_folds.csv')

sent_small_bert_L6_768 download started this may take some time.
Approximate size to download 240.9 MB
[OK!]


In [ ]:
x = bm.process()

ROUND 0 started
ROUND 0 completed, Time taken: 540.4855098724365 seconds


ROUND 1 started
ROUND 1 completed, Time taken: 522.435076713562 seconds


ROUND 2 started
ROUND 2 completed, Time taken: 511.38531374931335 seconds


ROUND 3 started
ROUND 3 completed, Time taken: 504.6034789085388 seconds


ROUND 4 started
ROUND 4 completed, Time taken: 503.52339363098145 seconds




In [ ]:
x

[{'accuracy': 0.55,
  'f1': 0.5523674854602701,
  'weighted precision': 0.5562822632156375,
  'weighted recall': 0.55},
 {'accuracy': 0.528,
  'f1': 0.5287430109696659,
  'weighted precision': 0.5298989409225998,
  'weighted recall': 0.528},
 {'accuracy': 0.529,
  'f1': 0.5279722125047982,
  'weighted precision': 0.5272582010327078,
  'weighted recall': 0.529},
 {'accuracy': 0.542,
  'f1': 0.5403536921995353,
  'weighted precision': 0.5401369311208017,
  'weighted recall': 0.5419999999999999},
 {'accuracy': 0.558,
  'f1': 0.5576061207793044,
  'weighted precision': 0.5573814029396452,
  'weighted recall': 0.5579999999999999}]